#### Amazon S3

In [1]:
import boto3
s3 = boto3.client('s3')

In [9]:
bucket_name = "itam-analytics-charlie"

full_path = "/workspaces/Arquitectura/"
file_paths = ["Tarea06/data/inegi/inflacion.csv", "Tarea06/data/banxico/tipo_cambio.csv", "Tarea06/data/banxico/tasa_interes.csv"]

file_paths = [full_path + file_path for file_path in file_paths]
for file_path in file_paths: 
    file_name = file_path.split("/")[-1]
    s3.upload_file(
        Filename=file_path,
        Bucket=bucket_name,
        Key="econ/raw/" + file_name.replace(".csv", "") + "/" + file_name
    )
    

#### Amazon Glue and Athena

In [10]:
from botocore.config import Config
my_config = Config(
    region_name = 'us-east-1'
)

glue = boto3.client('glue', config=my_config)

In [11]:
response = glue.create_database(
    DatabaseInput={
        'Name': 'econ',
        'Description': 'Base de datos de análisis de tipo de cambio, tasa de interés y de inflación',
    },
)
response

{'ResponseMetadata': {'RequestId': 'c10f28f2-15c7-4c9d-9c8a-bcaca566ddf5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Mar 2025 22:51:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c10f28f2-15c7-4c9d-9c8a-bcaca566ddf5',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0}}

In [12]:
import awswrangler as wr


query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_de_interes` (
    `date` DATE,
    `tasa_de_interes` DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-charlie/econ/raw/tasa_interes/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [13]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False
)

""


In [14]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio` (
    `date` DATE,
    `tipo_de_cambio` DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-charlie/econ/raw/tipo_cambio/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [15]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False
)

""


In [16]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion` (
    `date` DATE,
    `inflacion` DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES ('field.delim' = ',')
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 's3://itam-analytics-charlie/econ/raw/inflacion/'
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [17]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False
)

""
